Visualizing convnet filters

https://github.com/himanshurawlani/convnet-interpretability-keras/blob/master/Visualizing%20filters/visualizing_convnet_filters.ipynb

- for the gradients and : https://www.sicara.ai/blog/2019-08-28-interpretability-deep-learning-tensorflow
https://gist.github.com/RaphaelMeudec/31b7bba0b972ec6ec80ed131a59c5b3f#file-kernel_visualization-py

- for building blocks instead of layers (better visualization) as blocks (conv + pooling)
together can capture structures: https://github.com/nikhilroxtomar/Custom-Blocks-in-TensorFlow-using-Keras-API/blob/main/cifar10.py


In [ ]:
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
base_model = keras.applications.VGG19(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(50, 50, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, beta, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = beta
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        #self.beta_coefficient = beta_coefficient
    
    def call(self, inputs):
        x = self.encoder(inputs)[2]
        return self.decoder(x)
        
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    #removed tf.reduce_sum(MSE)
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #reconstruction_loss = tf.reduce_sum(keras.losses.MSE(data, reconstruction), axis=(1)) # mod 
            reconstruction_loss = np.prod((50, 50)) * tf.keras.losses.MSE(tf.keras.backend.flatten(data), tf.keras.backend.flatten(reconstruction)) # over weighted MSE    
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_sum(kl_loss, axis=1) #removed reduce_mean()
            
            total_loss = reconstruction_loss + (self.beta * kl_loss)
            total_loss = tf.reduce_mean(total_loss) 
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),x
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:

vae_encoder = load_model('models/vae_encoder.h5')
vae_decoder = load_model('models/vae_decoder.h5')
vae = VAE(encoder=vae_encoder, decoder=vae_decoder, beta = 1)
vae(np.zeros((1,50,50,3)))
vae.load_weights('weights/vae.h5')
model = vae_encoder

In [ ]:
for layer in model.layers:
    if 'conv' in layer.name:
        print(layer.name)
        print(len(layer.get_weights()))

Setting visualization variables

Maximize the activation of a specific filter

In [ ]:
import numpy as np
import tensorflow as tf

# Layer name to inspect
layer_name = 'block1_conv1'

epochs = 100
step_size = 1.
filter_index = 1

# Create a connection between the input and the target layer
submodel = tf.keras.models.Model([model.inputs[0]], [model.get_layer(layer_name).output])

# Initiate random noise
if 'encoder' in model.name:
    input_img_data = np.random.random((1, 50, 50, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128.

if 'decoder' in model.name:
    input_img_data = np.random.random((1, 512))
    input_img_data = (input_img_data - 0.5) * 20 + 128.

# Cast random noise from np.float64 to tf.float32 Variable
input_img_data = tf.Variable(tf.cast(input_img_data, tf.float32))

# Iterate gradient ascents
for _ in range(epochs):
    with tf.GradientTape() as tape:
        outputs = submodel(input_img_data)
        loss_value = tf.reduce_mean(outputs[:, :, :, filter_index])
    grads = tape.gradient(loss_value, input_img_data)
    normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
    input_img_data.assign_add(normalized_grads * step_size)

In [ ]:
img = input_img_data.numpy().astype(np.uint8)
img = img.squeeze()
img = img / 255
img.max()
img.shape

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
#dimensions of the generated pictures for each filter.
img_width = 50
img_height = 50

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
#layer_dict = dict([(layer.name, layer) for layer in model.layers[0:]])
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
layer_dict

In [ ]:
from keras import backend as K

# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [ ]:
def gradient_ascent(iterate):
    # step size for gradient ascent    
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

#         print('------>Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break
        
    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))

In [ ]:
def build_nth_filter_loss(filter_index, layer_name):
    """
    We build a loss function that maximizes the activation
    of the nth filter of the layer considered
    """
    
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])
    # Initiate random noise
    # Create a connection between the input and the target layer
    
    submodel = tf.keras.models.Model([model.inputs[0]], [model.get_layer(layer_name).output])

# Initiate random noise

    input_img_data = np.random.random((1, 50, 50, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128.

    # Cast random noise from np.float64 to tf.float32 Variable
    input_img_data = tf.Variable(tf.cast(input_img_data, tf.float32))

    for _ in range(epochs):
        with tf.GradientTape() as tape:
            outputs = submodel(input_img_data)
            loss_value = tf.reduce_mean(outputs[:, :, :, filter_index])
        grads = tape.gradient(loss_value, input_img_data)
        normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
        input_img_data.assign_add(normalized_grads * step_size)

    # this function returns the loss and grads given the input picture
    #iterate = K.function([input_img], [loss_value, grads])

    if loss_value > 0:
        img = input_img_data.numpy().astype(np.float64)
        img = img.squeeze()
        img = deprocess_image(img)
        kept_filters.append((img, loss_value))
    #return iterate

In [ ]:
layers = [layer.name for layer in model.layers]

In [ ]:
layer = model.get_layer('block1_conv1')
range(min(layer.output.shape[-1], 100))
layer_name

In [ ]:
filter_index = 5
build_nth_filter_loss(filter_index, layer_name)

In [ ]:
import time

kept_filters = []
filters_dict = dict()
for layer_name in layers:
    if 'conv' in layer_name:
        layer = model.get_layer(layer_name)
        print('Processing filter for layer:', layer_name)
        for filter_index in range(min(layer.output.shape[-1], 100)):
            # print('Processing filter %d' % filter_index)

            start_time = time.time()
            build_nth_filter_loss(filter_index, layer_name)
            end_time = time.time()

    #         print('--->Filter %d processed in %ds' % (filter_index, end_time - start_time))
        filters_dict[layer.name] = kept_filters
        kept_filters = []

In [ ]:
for layer_name, kept_filters in filters_dict.items():
    print(layer_name, len(kept_filters))

In [ ]:
from keras.preprocessing.image import save_img

def stich_filters(kept_filters, layer_name):
    # By default, we will stich the best 64 (n*n) filters on a 8 x 8 grid.
    n = int(np.sqrt(len(kept_filters)))
    # the filters that have the highest loss are assumed to be better-looking.
    # we will only keep the top 64 filters.
    kept_filters.sort(key=lambda x: x[1], reverse=True)
    kept_filters = kept_filters[:n * n]

    # build a black picture with enough space for
    # our 8 x 8 filters of size 128 x 128, with a 5px margin in between
    margin = 5
    width = n * img_width + (n - 1) * margin
    height = n * img_height + (n - 1) * margin
    stitched_filters = np.zeros((width, height, 3))

    # fill the picture with our saved filters
    for i in range(n):
        for j in range(n):
            img, loss = kept_filters[i * n + j]
            width_margin = (img_width + margin) * i
            height_margin = (img_height + margin) * j
            stitched_filters[
                width_margin: width_margin + img_width,
                height_margin: height_margin + img_height, :] = img

    # save the result to disk
    save_img('img/filters/vae/{}_stitched_filters_{}.png'.format(model.name, layer_name), stitched_filters)
    
for layer_name, kept_filters in filters_dict.items():
    print('Stiching filters for {}'.format(layer_name))
    stich_filters(kept_filters, layer_name)
    print('number of filters kept:', len(kept_filters))
    print('Completed.')

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

filter_name = 'block1_conv1'

img = image.img_to_array(image.load_img('img/filters/vae/{}_stitched_filters_{}.png'.format(model.name, filter_name))) /255.
plt.figure(figsize=(17,17))
plt.imshow(img)
plt.title(filter_name)
plt.grid(False)